# Bilibili Download

In [ ]:
import requests
import json
import re
from pprint import pprint

In [ ]:
# F12"检查" -- Network -- Name下面的一栏 -- Headers -- User-Agent

In [ ]:
url = r'https://www.bilibili.com/video/BV1DS421X7s3/'

header = {
  "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
  "Referer":"https://www.bilibili.com/"
}

response = requests.get(url, headers=header)
# print(response.status_code)
# print(response.text)

In [ ]:
obj = re.compile(r'window.__playinfo__=(.*?)</script>', re.S)
html_data = obj.findall(response.text)[0]
# print(html_data)
json_data = json.loads(html_data)
# pprint(json_data)

# videos = json_data['data']['dash']['video']
# video_url = videos[0]['baseUrl']
# resp1 = requests.get(url=video_url, headers=header)

# with open('test.mp4', mode = 'wb') as f:
#   f.write(resp1.content)


audio = json_data['data']['dash']['audio']
audio_url = audio[0]['baseUrl']
resp2 = requests.get(url=audio_url, headers=header)

with open('test.mp3', mode = 'wb') as f:
  f.write(resp2.content)

# VIDEO SUMMARY

### Install & Import Libraries

In [ ]:
# Install packages.
!pip install srt==3.5.3
!pip install datasets==2.20.0
!pip install DateTime==5.5
!pip install OpenCC==1.1.7
!pip install opencv-contrib-python==4.8.0.76
!pip install opencv-python==4.8.0.76
!pip install opencv-python-headless==4.10.0.84
!pip install openpyxl==3.1.4
!pip install openai==1.35.3
!pip install git+https://github.com/openai/whisper.git@ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
!pip install numpy==1.25.2
!pip install soundfile==0.12.1
!pip install -q -U google-generativeai==0.7.0
!pip install anthropic==0.29.0

In [ ]:
# Import packages.
import whisper
import srt
import datetime
import time
import os
import re
import pathlib
import textwrap
import numpy as np
import soundfile as sf
from opencc import OpenCC
from tqdm import tqdm
from datasets import load_dataset
from openai import OpenAI
import google.generativeai as genai
import anthropic

### Load Data

In [ ]:
import librosa as librosa

# 替换'path_to_your_audio_file.wav'为你的音频文件路径
audio_path = '/content/test.mp3'
input_audio_array, sample_rate = librosa.load(audio_path)

<ipython-input-2-2d71085dcaac>:5: UserWarning: PySoundFile failed. Trying audioread instead.
  input_audio_array, sample_rate = librosa.load(audio_path)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [ ]:
# # 从 HuggingFace load audio
# dataset_name = "kuanhuggingface/NTU-GenAI-2024-HW9"
# dataset = load_dataset(dataset_name)

# input_audio = dataset["test"]["audio"][0]
# input_audio_array = input_audio["array"].astype(np.float32) # 1d array

### 1 Audio to Text with Whisper

In [ ]:
# WHISPER Parameters
whisper_options = {
    "model_name": "medium", # ["tiny", "base", "small", "medium", "large-v3"]
    "output_subtitle_path": "./output.txt",
    "language": "zh",
    "initial_prompt": "請用简体中文",
    "temperature": 0.0,
    "cache_dir": "./",
    "model_dir": "/content/medium.pt"
}

In [ ]:
# You can see "https://github.com/openai/whisper/blob/main/whisper/decoding.py" and "https://github.com/openai/whisper/blob/main/whisper/transcribe.py" for more details.

# WHISPER Speech Recognition
# def speech_recognition(whisper_options, input_audio_array):
print(f"=============== Loading Model ===============")
if os.path.exists(whisper_options['model_dir']):
  model = whisper.load_model(whisper_options['model_dir'])
else:
  model = whisper.load_model(name=whisper_options["model_name"], download_root = whisper_options['cache_dir'])

print(f"=============== Transcribe Audio ===============")
transcription = model.transcribe(audio=input_audio_array,
                language=whisper_options["language"],
                verbose=False,
                initial_prompt = whisper_options["initial_prompt"],
                temperature = whisper_options["temperature"])

print(f"=============== Saving to TXT ===============")
with open(whisper_options['output_subtitle_path'], 'w', encoding='utf-8') as f:
    f.write(transcription['text'])

=============== Loading Model ===============


100%|█████████████████████████████████████| 1.42G/1.42G [00:17<00:00, 88.2MiB/s]


=============== Transcribe Audio ===============


100%|██████████| 158934/158934 [04:28<00:00, 591.61frames/s]

=============== Saving to TXT ===============


In [ ]:
with open(whisper_options['output_subtitle_path'], 'r', encoding='utf-8') as f:
    content = f.read()

print(f"=============== Split into Chunks ===============")
chunks = textwrap.wrap(transcription['text'], width = 512)
chunks

### 2 GPT Summary

In [ ]:
# OpenAI Parameters
openai_api_key = "YOUR_API_KEY"

gpt_options = {
    "model_name": "gpt-3.5-turbo",
    "temperature": 0.0,
    "top_p": 1.0,
    "max_tokens": 512,
    "client": OpenAI(api_key=openai_api_key),
    "summarization_prompt_template": "請在 500 字以內，提供以下文字的簡潔摘要:<text>",
    "summarization_prompt_refinement_template": "請在 500 字以內，結合原先的摘要和新的內容，提供簡潔的摘要:<text>"
}

In [ ]:
# MULTI-STAGE SUMMARIZATION

chunk_summary_list = []

for i in range(len(chunks)):
  user_prompt = gpt_options["summarization_prompt_template"].replace("<text>", chunks[i])

  chat_completion = gpt_options['client'].chat.completions.create(
              messages = [{"role": "user", "content": user_prompt}],
              model = gpt_options['model_name'],
              temperature = gpt_options['temperature'],
              top_p = gpt_options['top_p'],
              max_tokens = gpt_options['max_tokens'])

  chunk_summary = chat_completion.choices[0].message.content
  print(f"Chunk {i} summary: {chunk_summary}")
  chunk_summary_list.append(chunk_summary)

chunk_summary_concat = ' '.join(chunk_summary_list)
chunk_summary_prompt = gpt_options["summarization_prompt_template"].replace("<text>", chunk_summary_concat)

chat_completion = gpt_options['client'].chat.completions.create(
            messages = [{"role": "user", "content": chunk_summary_prompt}],
            model = gpt_options['model_name'],
            temperature = gpt_options['temperature'],
            top_p = gpt_options['top_p'],
            max_tokens = gpt_options['max_tokens'])
chat_completion.choices[0].message.content